In [ ]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT
#from qiskit.circuit.library import ModularExponentiation
#from qiskit.circuit.library import ModularMultiplicationGate
from math import gcd, ceil, log
from fractions import Fraction
import numpy as np

ImportError: cannot import name 'ModularMultiplicationGate' from 'qiskit.circuit.library' (/opt/anaconda3/envs/qiskit/lib/python3.10/site-packages/qiskit/circuit/library/__init__.py)

In [ ]:

def Uf(circ):
    for i in range(2*n):
        base = int(a)
        exp = int(2**i)
        mod = int(N)
        mod_exp = pow(base, exp, mod)
        circ.cp(2*np.pi*float(mod_exp)/float(N), q[i], t)

In [82]:
def apply_c_amodX(qc, a: int, q: int, N: int, n_count: int, t) -> QuantumCircuit:
    mod_exp = pow(a, 2**q, N)
    #qc.cp(2*np.pi*mod_exp/N, q, t)
    qc.cp(2*np.pi*mod_exp/N, q, t)
    return qc

In [10]:
def apply_c_amod15(qc, a: int, control_qubit: int, n_count: int) -> QuantumCircuit:
    x = n_count
    if a == 2:
        qc.cswap(control_qubit, x, x+1)
        qc.cswap(control_qubit, x+1, x+2)
        qc.cswap(control_qubit, x+2, x+3)
    elif a == 4:
        qc.cswap(control_qubit, x, x+2)
        qc.cswap(control_qubit, x+1, x+3)
    elif a == 7:
        qc.cswap(control_qubit, x, x+3)
        qc.cswap(control_qubit, x+1, x+2)
    elif a == 8:
        qc.cswap(control_qubit, x, x+2)
        qc.cswap(control_qubit, x+1, x+3)
        qc.cswap(control_qubit, x+2, x+3)
    elif a == 11:
        qc.cswap(control_qubit, x, x+1)
        qc.cswap(control_qubit, x+1, x+2)
        qc.cswap(control_qubit, x+2, x+3)
    elif a == 13:
        qc.cswap(control_qubit, x, x+2)
        qc.cswap(control_qubit, x+1, x+3)
        qc.cswap(control_qubit, x+2, x+3)
    return qc

In [94]:
def qpe_modexp0(a: int, N: int, n_count: int) -> QuantumCircuit:
    n = n_count
    
    """ Create quantum and classical registers """
    q = QuantumRegister(2*n)
    t = QuantumRegister(n)
    c = ClassicalRegister(2*n)


    """ Create Quantum Circuit """
    qc = QuantumCircuit(q,t,c)

    """ Initialize down register to 1"""
    qc.x(t)
    qc.h(q)

 
    #for q in range(2*n_count):
        #qc = apply_c_amod15(qc, a**(2**q) % N, q, n_count)
    #    qc = apply_c_amodX(qc, a, q, N, n_count, tr)

    for i in range(2*n_count):
        base = int(a)
        exp = int(2**i)
        mod = int(N)
        mod_exp = pow(base, exp, mod)
        qc.cp(2*np.pi*float(mod_exp)/float(N), q[i], t)

    qc.append(QFT(num_qubits=n_count, inverse=True, do_swaps=True), range(n_count))
    qc.measure(q, c)
    return qc

In [102]:
def qpe_modexp(a: int, N: int, n_count: int) -> QuantumCircuit:
    n = n_count
    


    qc = QuantumCircuit(2*n_count, n_count)
    for q in range(n_count):
        qc.h(q)
    qc.x(n_count)
    for q in range(n_count):
        qc = apply_c_amod15(qc, a**(2**q) % N, q, n_count)

    qc.append(QFT(num_qubits=n_count, inverse=True, do_swaps=True), range(n_count))
    qc.measure(range(n_count), range(n_count))
    return qc

In [55]:
def get_random_base(N):
    base = np.random.randint(2, N)
    while gcd(base, N) != 1:
        base = np.random.randint(2, N)
    return base

In [56]:
def get_period(phase: int, n_count: int) -> int:
    decimal = phase / (2 ** n_count)
    frac = Fraction(decimal).limit_denominator(15)
    return frac.denominator

In [119]:
def shor_manual(N: int, a: int = 7) -> int:
    if gcd(a, N) != 1:
        return gcd(a, N)

    n_count = N.bit_length() # get run only for min bits

    qc = qpe_modexp(a, N, n_count)

    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit)
    result = job.result()
    
    counts = result.get_counts()
    print(counts)
    phase_bin = max(counts, key=counts.get)
    print(phase_bin)
    phase_int = int(phase_bin, 2)
    r = get_period(phase_int, n_count)
    print(r)
    if r % 2 != 0:
        return None
    plus = pow(a, r // 2) + 1
    minus = pow(a, r // 2) - 1
    factor1 = gcd(plus, N)
    factor2 = gcd(minus, N)
    if factor1 == 1 or factor1 == N:
        return None
    return factor1

In [120]:
N = 15
a = get_random_base(N)
factor = None
while factor == None :
    a = get_random_base(N)
    factor = shor_manual(N, a)
print(f"Found factor of {N} using base {a}: {factor}")

{'1100': 263, '1000': 234, '0000': 286, '0100': 241}
0000
1
{'1100': 262, '0100': 259, '0000': 259, '1000': 244}
1100
4
Found factor of 15 using base 8: 5


In [84]:
N = 15
a = get_random_base(N)
factor = None
while factor == None :
    a = get_random_base(N)
    factor = shor_manual(N, a)
print(f"Found factor of {N} using base {a}: {factor}")

CircuitError: 'Bit \'<Qubit register=(4, "q27"), index=0>\' is not in the circuit.'

In [52]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Gate
import numpy as np

def is_unitary(U, tol=1e-10):
    # QR decomposition gives a unitary Q
    Q, R = np.linalg.qr(U)

    # Now Q is unitary: Q†Q = I
    #print(np.allclose(Q.conj().T @ Q, np.eye(4)))  # Should be True

    #return np.allclose(Q.conj().T @ Q, np.eye(Q.shape[0]), atol=tol)
    return Q

def create_modular_multiplication_gate(a, N, n):
    """
    Create a hard-coded modular multiplication gate for small a and N.
    This assumes inputs are in |y⟩ and maps to |a*y mod N⟩.
    """
    dim = 2**n
    U = np.zeros((dim, dim), dtype=complex)

    for y in range(dim):
        result = (a * y) % N
        U[result][y] = 1.0  # Basis map: |y⟩ → |(a*y) mod N⟩

    print(is_unitary(U))
    U =  is_unitary(U)
    modmul_gate = Gate(name=f"ModMul_{a}_mod_{N}", num_qubits=n, params=[])
    qc = QuantumCircuit(n)
    qc.unitary(U, list(range(n)), label=f"{a}*y mod {N}")
    return qc.to_gate(label=f"{a}*y mod {N}")

# Example usage
a = 7
N = 15
n = 4  # Number of bits to hold values up to N

mod_mul_gate = create_modular_multiplication_gate(a, N, n)

# Use in a circuit
qr = QuantumRegister(n)
qc = QuantumCircuit(qr)
qc.append(mod_mul_gate, qr)
#qc.draw('mpl')


[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.-0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.-0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.-0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.-0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.-0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.-0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.

In [64]:

N = 15 # number to factorize
N = int(N)
#n = ceil(log2(N))  # number of bits to represent N
n = N.bit_length()
q = QuantumRegister(2*n, "q")
t = QuantumRegister(n, "t")
c = ClassicalRegister(2*n, "c")
#state = False
#service = QiskitRuntimeService()


def Uf(circ):
    for i in range(2*n):
        base = int(a)
        exp = int(2**i)
        mod = int(N)
        mod_exp = pow(base, exp, mod)
        circ.cp(2*np.pi*float(mod_exp)/float(N), q[i], t)



In [131]:

#a = random.randint(2, N-1)
a = 2
while gcd(a, N) > 1:
    a = random.randint(2, N-1)

print(f"Chosen base a = {a}")


qpe = QuantumCircuit(q, t, c)
qpe.x(t)
qpe.barrier()
qpe.h(q)
Uf(qpe)
qpe.append(QFT(2*n, inverse=True), q)
qpe.measure(range(n*2), range(n*2))
qpe.barrier()
qpe.measure(q, c)

simulator = AerSimulator()
compiled_circuit = transpile(qpe, simulator)
job = simulator.run(compiled_circuit)
result = job.result()
    
counts = result.get_counts()
#print(counts)


sorted_measurements = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    
for measurement, _ in sorted_measurements:
    decimal_value = int(measurement, 2)
    #r = retrieve_denominator(decimal_value, n, N)
    r = get_period(decimal_value,n)
    print(decimal_value)
    if isinstance(r, int) and r % 2 == 0:
        x = pow(a, r // 2, N)
        factors = (gcd(x - 1, N), gcd(x + 1, N))
        
        if (factors[0] != 1 or factors[1] != 1) and (factors[0] != N or factors[1] != N):
            if factors[0] * factors[1] == N:
                print(f"The factors for {N} are: {factors[0]}, {factors[1]}")
                state = True
                break
            elif factors[0] != 1 and ((N // factors[0]) * factors[0] == N):
                print(f"The factors for {N} are: {factors[0]}, {N // factors[0]}")
                state = True
                break
            elif factors[1] != 1 and ((N // factors[1]) * factors[1] == N):
                print(f"The factors for {N} are: {factors[1]}, {N // factors[1]}")
                state = True
                break
            

Chosen base a = 2
145
137
146
The factors for 15 are: 15, 1


In [127]:

def retrieve_denominator(decimal_value, n, N):
    decimal_value = int(decimal_value)
    n = int(n)
    N = int(N)
    
    url = "http://127.0.0.1:5000/denominator"
    payload = {
        "decimal_value": decimal_value,
        "n": n,
        "N": N
    }
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            return response.json().get("denominator")
        else:
            return f"Error: {response.status_code}, {response.json().get('error', 'Unknown error')}"
    except Exception as e:
        return f"Request failed: {str(e)}"

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.circuit.library import QFT
from fractions import Fraction
from math import gcd

def shors_algorithm(N, a, n=4, shots=1024):
    """Run a simplified Shor’s algorithm on N using base a and n qubits."""
    if gcd(a, N) != 1:
        return f"Trivially found factor {gcd(a, N)}"
    
    # Quantum part
    qc = QuantumCircuit(2 * n, n)

    # Step 1: Hadamards on input register
    for q in range(n):
        qc.h(q)

    # Step 2: Compute a^x mod N into output register — HARD CODED
    for x_val in range(2 ** n):
        result = pow(a, x_val, N)
        binary = format(result, f'0{n}b')
        qc_temp = QuantumCircuit(2 * n)
        for i, bit in enumerate(format(x_val, f'0{n}b')):
            if bit == '0':
                qc_temp.x(i)
        for j, bit in enumerate(binary):
            if bit == '1':
                qc_temp.x(n + j)
        for i, bit in enumerate(format(x_val, f'0{n}b')):
            if bit == '0':
                qc_temp.x(i)
        qc = qc.compose(qc_temp)

    # Step 3: Inverse QFT
    qc.append(QFT(n, inverse=True, do_swaps=True).to_gate(), list(range(n)))

    # Step 4: Measure input register
    qc.measure(list(range(n)), list(range(n)))

    # Simulate
    sim = Aer.get_backend("aer_simulator")
    qc = transpile(qc, sim)
    qobj = assemble(qc, shots=shots)
    result = sim.run(qobj).result()
    counts = result.get_counts()

    print("Measurement counts:", counts)

    # Classical post-processing
    for outcome in sorted(counts, key=counts.get, reverse=True):
        x_val = int(outcome, 2)
        r = find_period(x_val, n, a, N)
        if r:
            factors = try_factors(a, r, N)
            if factors:
                return f"Success: factors of {N} are {factors}"
    return "Failed to find factors"

def find_period(measured_value, n, a, N, max_denominator=32):
    """Estimate r from measured_value / 2^n."""
    frac = Fraction(measured_value, 2 ** n).limit_denominator(max_denominator)
    r = frac.denominator
    print(f"Trying r = {r} from continued fraction {frac}")
    if pow(a, r, N) == 1:
        return r
    for k in range(2, 6):
        r_try = r * k
        if pow(a, r_try, N) == 1:
            print(f"Found valid r = {r_try} (multiple of {r})")
            return r_try
    return None

def try_factors(a, r, N):
    """Try to compute the factors using r."""
    if r % 2 != 0:
        return None
    plus = pow(a, r // 2) + 1
    minus = pow(a, r // 2) - 1
    f1 = gcd(plus, N)
    f2 = gcd(minus, N)
    if 1 < f1 < N and 1 < f2 < N:
        return (f1, f2)
    return None
